want to compare the increase of cabi rides between maryland and dc

2022,2023,2024

steps
filter the data set for stations and rides in dc and maryland:
1) take the complete dataset, turn it into geodataframe
2) join it with maryland gdf
3) make plot to compare the difference of growth between them

dive deeper into maryland / prince george county for 2023 -2024

In [2]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import statistics as st
import seaborn as sns
import datetime 
from geopy import distance
import folium
from folium.plugins import MarkerCluster
from folium.features import GeoJsonTooltip
from branca.colormap import LinearColormap
from collections import Counter
import json
from shapely.geometry import Point
import geopandas as gpd
from shapely.geometry import shape

In [3]:
data_types = {
    "rideable_type": "category", 
    "start_station_name": "category", 
    "end_station_name": "category", 
    "member_casual":"category",
    # "ride_id":"uint32",
    "time_of_day":"category",
    "trip_type":"category"}

In [4]:
bikes = pd.read_csv("bikes_clean.csv",parse_dates= ["started_at", "ended_at"], low_memory=False)

In [5]:
cabi_2024 = pd.read_csv("cabi_2024.csv")

In [7]:
cabi_updated = pd.concat([bikes, cabi_2024],ignore_index=True,axis=0)

In [8]:
cabi_updated.shape

(16808320, 20)

In [9]:
# # Create a GeoDataFrame for the start stations
geometry = [Point(xy) for xy in zip(cabi_updated['start_lng'], cabi_updated['start_lat'])]
cabi_updated_gdf = gpd.GeoDataFrame(cabi_updated, geometry=geometry, crs="EPSG:4326")



In [11]:
cabi_updated_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 16808320 entries, 0 to 16808319
Data columns (total 21 columns):
 #   Column                 Dtype   
---  ------                 -----   
 0   ride_id                object  
 1   rideable_type          object  
 2   started_at             object  
 3   ended_at               object  
 4   start_station_name     object  
 5   start_station_id       object  
 6   end_station_name       object  
 7   end_station_id         object  
 8   member_casual          object  
 9   start_lat              float64 
 10  start_lng              float64 
 11  end_lat                float64 
 12  end_lng                float64 
 13  start_time             object  
 14  end_time               object  
 15  start_hour             float64 
 16  end_hour               float64 
 17  trip_duration_minutes  float64 
 18  time_of_day            object  
 19  Unnamed: 0             float64 
 20  geometry               geometry
dtypes: float64(8), geomet

In [12]:
cabi_updated_gdf['ride_id'] = pd.Series(range(1, len(bikes) + 1), dtype='uint32')

In [15]:
cabi_updated_gdf.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,member_casual,start_lat,...,end_lat,end_lng,start_time,end_time,start_hour,end_hour,trip_duration_minutes,time_of_day,Unnamed: 0,geometry
0,1.0,classic_bike,2021-01-01 00:08:33,2021-01-01 00:33:53,Maine Ave & 9th St SW,31646,Rosslyn Metro / Wilson Blvd & Ft Myer Dr,31015,member,38.880440,...,38.894600,-77.072305,00:08:33,00:33:53,0.0,0.0,25.0,night,NaN,POINT (-77.02524 38.88044)
1,2.0,electric_bike,2021-01-01 00:13:43,2021-01-01 00:29:34,10th & U St NW,31111,NaN,31515,casual,38.917193,...,38.960000,-77.020000,00:13:43,00:29:34,0.0,0.0,16.0,night,NaN,POINT (-77.02589 38.91719)
2,3.0,electric_bike,2021-01-01 00:14:32,2021-01-01 00:28:45,17th & Corcoran St NW,31214,14th & Belmont St NW,31119,member,38.912138,...,38.920870,-77.031691,00:14:32,00:28:45,0.0,0.0,14.0,night,NaN,POINT (-77.03857 38.91214)
3,4.0,classic_bike,2021-01-01 00:15:45,2021-01-01 00:21:20,Wilson Blvd. & N. Vermont St.,31926,Wilson Blvd. & N. Vermont St.,31926,member,38.879477,...,38.879477,-77.114563,00:15:45,00:21:20,0.0,0.0,6.0,night,NaN,POINT (-77.11456 38.87948)
4,5.0,classic_bike,2021-01-01 00:17:46,2021-01-01 00:21:00,11th & Park Rd NW,31651,14th & Newton St NW,31649,member,38.931322,...,38.931991,-77.032956,00:17:46,00:21:00,0.0,0.0,3.0,night,NaN,POINT (-77.02825 38.93132)


In [22]:
cabi_updated_gdf['started_at'] = pd.to_datetime(cabi_updated_gdf['started_at'])#, format='%Y-%m-%d %H:%M:%S')

In [23]:
cabi_updated_gdf['ended_at'] = pd.to_datetime(cabi_updated_gdf['ended_at'])

In [24]:
cabi_updated_gdf['year'] = cabi_updated_gdf['started_at'].dt.year

In [27]:
cabi_updated_gdf['month'] = cabi_updated_gdf['started_at'].dt.month

In [26]:
cabi_updated_gdf.to_csv("cabi_updated_gdf.csv")

In [28]:
# cabi updated gdf sample 

# 108.000 rows sample (3000 per month)
samples = []

# Loop over years and months to sample 1,000 rows for each month
for year in range(2021, 2025):  # Covering 2021, 2022, and 2024
    for month in range(1, 13):  # Months from 1 to 12
        # Filter for the specific year and month
        filtered_data = cabi_updated_gdf[(cabi_updated_gdf["year"] == year) & (cabi_updated_gdf["month"] == month)]
        
        # Sample 1,000 rows if there are enough rows available
        if len(filtered_data) >= 3000:
            sample = filtered_data.sample(n=3000, random_state=42)
        else:  # If less than 3,000 rows, take all available rows
            sample = filtered_data
            
        samples.append(sample)

# Combine all the samples into a single DataFrame
bikes_monthly_sample_3000 = pd.concat(samples, ignore_index=True)

In [29]:
bikes_monthly_sample_3000.to_csv("bikes_sample_updated.csv", index=False)

In [30]:
# uploading ward info so I can filter out DC area
with open ("Wards_from_2022.geojson") as i:
    wards = json.loads(i.read())

features =wards["features"]

ward_gdf = gpd.GeoDataFrame(
    pd.DataFrame([feature['properties'] for feature in features]),  # Extract properties as attributes
    geometry=[shape(feature['geometry']) for feature in features],  # Convert geometries
    crs="EPSG:4326" )

In [31]:
dc_wards_gdf = gpd.sjoin(bikes_monthly_sample_3000, ward_gdf, how="left", predicate="within")

In [38]:
dc_wards_gdf = dc_wards_gdf[['ride_id',
 'rideable_type',
 'started_at',
 'ended_at',
 'start_station_name',
 'start_station_id',
 'end_station_name',
 'end_station_id',
 'member_casual',
 'start_lat',
 'start_lng',
 'end_lat',
 'end_lng',
 'start_time',
 'end_time',
 'start_hour',
 'end_hour',
 'trip_duration_minutes',
 'time_of_day',
 'Unnamed: 0',
 'geometry',
 'year',
 'month',
 'WARD',"NAME","SHAPEAREA","SHAPELEN"]]

In [39]:
with open ("Maryland_Physical_Boundaries_-_County_Boundaries_(Detailed).geojson") as i:
    maryland = json.loads(i.read())

features = maryland["features"]

#GDF
maryland_gdf = gpd.GeoDataFrame(
    pd.DataFrame([feature['properties'] for feature in features]),  # Extract properties as attributes
    geometry=[shape(feature['geometry']) for feature in features],  # Convert geometries
    crs="EPSG:4326")
    

In [40]:
# # Perform spatial join with the maryland gdf
dc_maryland_gdf= gpd.sjoin(dc_wards_gdf, maryland_gdf, how="left", predicate="within")

In [ ]:
dc_maryland_gdf[["NAME","COUNTY"]]

,NAME,COUNTY
0,Ward 2,NaN
1,NaN,Prince George's
2,Ward 1,NaN
3,Ward 1,NaN
4,NaN,NaN
...,...,...
143995,Ward 4,NaN
143996,Ward 5,NaN
143997,NaN,NaN
143998,NaN,NaN


In [47]:
dc_maryland_gdf = dc_maryland_gdf[dc_maryland_gdf["NAME"].notnull()| dc_maryland_gdf["COUNTY"].notnull()]

In [49]:
dc_maryland_gdf.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'member_casual', 'start_lat', 'start_lng', 'end_lat',
       'end_lng', 'start_time', 'end_time', 'start_hour', 'end_hour',
       'trip_duration_minutes', 'time_of_day', 'Unnamed: 0', 'geometry',
       'year', 'month', 'WARD', 'NAME', 'SHAPEAREA', 'SHAPELEN', 'index_right',
       'OBJECTID', 'COUNTY', 'DISTRICT', 'COUNTY_FIP', 'COUNTYNUM',
       'CREATION_D', 'LAST_UPDAT', 'Shape_Length', 'Shape_Area'],
      dtype='object')

In [53]:
dc_maryland_gdf_sample = dc_maryland_gdf[['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'member_casual', 'start_lat', 'start_lng', 'end_lat',
       'end_lng', 'start_time', 'end_time', 'start_hour', 'end_hour',
       'trip_duration_minutes', 'time_of_day', 'geometry',
       'year', 'month', 'WARD', 'NAME',"COUNTY"]]

In [54]:
dc_maryland_gdf_sample.to_csv("dc_maryland_gdf_sampled.csv")